In [4]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# MySQL 설정
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", 3306))
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
SERVICE_KEY = os.getenv("SERVICE_KEY")

# 연결 객체
conn = None
# MySQL 연결
def create_connection():
    """MySQL 연결을 생성하고 반환."""
    try:
        return mysql.connector.connect(
            host=DB_HOST,
            port=DB_PORT,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
            pool_name="mypool",
            pool_size=5
        )
    except mysql.connector.Error as e:
         return None


conn = None

def ensure_connection():
    global conn
    if conn is None or not conn.is_connected():
        print("Database connection is not active. Attempting to reconnect...")
        conn = create_connection()
        if conn and conn.is_connected():
            print("Reconnected to the database.")
        else:
            print("Failed to reconnect to the database.")

import pandas as pd

# MySQL 연결 확인 및 데이터 읽기
ensure_connection()

if conn and conn.is_connected():
    #df1 = pd.read_sql("SELECT * FROM oceandata;", conn)
    df2 = pd.read_sql("SELECT * FROM aisdatatest;", conn)

    # CSV 저장
    #df1.to_csv('../document/oceandata.csv', index=False, encoding='utf-8-sig')
    #print("데이터가 'oceandata.csv'로 저장되었습니다.")
    df2.to_csv('../document/aisdata.csv', index=False, encoding='utf-8-sig')
    print("데이터가 'aisdata.csv'로 저장되었습니다.")
else:
    print("Failed to connect to the database.")


Database connection is not active. Attempting to reconnect...
Reconnected to the database.


C:\Users\user\AppData\Local\Temp\ipykernel_8740\3858158761.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql("SELECT * FROM aisdatatest;", conn)


데이터가 'aisdata.csv'로 저장되었습니다.


In [8]:
import pandas as pd
ocean_data = pd.read_csv('../document/ocean.csv')

ocean_data

,datetime,wind_direct,wind_speed,surface_curr_drc,surface_curr_speed,air_temperature,water_temperature,air_pressure,humidity,salinity
0,2025-01-21 11:20,343,2.06,96,0.1,6.7,12.7,1019,48,19.5
1,2025-01-21 11:10,337,3.08,99,0.2,6.9,12.7,1019,48,19.5
2,2025-01-21 11:00,330,3.60,78,0.2,6.9,12.7,1019,48,19.4
3,2025-01-21 10:50,354,3.60,97,0.2,6.7,12.7,1019,51,19.5
4,2025-01-21 10:40,348,4.11,97,0.3,6.5,12.7,1019,53,19.5
...,...,...,...,...,...,...,...,...,...,...
202,2025-01-20 0:40,179,2.57,61,0.2,7.5,12.7,1013,35,20.8
203,2025-01-20 0:30,169,2.57,44,0.1,7.7,12.6,1013,38,20.6
204,2025-01-20 0:20,169,3.08,59,0.1,7.8,12.6,1013,40,-
205,2025-01-20 0:10,158,2.57,35,0.1,7.9,12.6,1013,35,21.8


In [32]:
import pandas as pd

# 원본 데이터 로드
data = pd.read_csv('../document/ocean.csv')

# datetime 형식으로 변환
data['datetime'] = pd.to_datetime(data['datetime'])

# 전체 시간 범위 생성
start_time = data['datetime'].min()
end_time = data['datetime'].max()
full_time_range = pd.date_range(start=start_time, end=end_time, freq='10T')

# 누락된 시간 구간 확인
missing_times = pd.DataFrame({'datetime': full_time_range}).merge(data, on='datetime', how='left')
missing_rows = missing_times[missing_times.isna().any(axis=1)]

print("누락된 시간 구간:")
print(missing_rows['datetime'])


누락된 시간 구간:
88    2025-01-20 14:40:00
101   2025-01-20 16:50:00
162   2025-01-21 03:00:00
163   2025-01-21 03:10:00
175   2025-01-21 05:10:00
205   2025-01-21 10:10:00
Name: datetime, dtype: datetime64[ns]


C:\Users\user\AppData\Local\Temp\ipykernel_8740\3146822849.py:12: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_time_range = pd.date_range(start=start_time, end=end_time, freq='10T')


In [34]:
# 보간 과정 점검
data = data.set_index('datetime').sort_index()  # datetime을 인덱스로 설정
data = data.interpolate(method='time')  # 시간 기반 보간
data = data.reset_index()  # 인덱스를 다시 열로 복원

# 누락된 시간 구간 데이터프레임 생성
full_time_range = pd.date_range(start=data['datetime'].min(), end=data['datetime'].max(), freq='10T')
full_data = pd.DataFrame({'datetime': full_time_range})

# 원본 데이터와 병합하여 누락된 시간 채우기
merged_data = full_data.merge(data, on='datetime', how='left')

# 결측치 처리 (앞의 값으로 채우기)
merged_data = merged_data.fillna(method='ffill')  # 이전 값으로 채움
# 누락된 시간 구간 확인
missing_times = pd.DataFrame({'datetime': full_time_range}).merge(merged_data, on='datetime', how='left')
missing_rows = missing_times[missing_times.isna().any(axis=1)]

print("남아 있는 누락된 시간 구간:")
print(missing_rows['datetime'])

# **CSV 파일로 저장**
merged_data.to_csv('../document/corrected_oceandata.csv', index=False)

print("데이터가 'corrected_data.csv'로 저장되었습니다.")

남아 있는 누락된 시간 구간:
Series([], Name: datetime, dtype: datetime64[ns])
데이터가 'corrected_data.csv'로 저장되었습니다.


C:\Users\user\AppData\Local\Temp\ipykernel_8740\66179513.py:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='time')  # 시간 기반 보간
C:\Users\user\AppData\Local\Temp\ipykernel_8740\66179513.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_time_range = pd.date_range(start=data['datetime'].min(), end=data['datetime'].max(), freq='10T')
C:\Users\user\AppData\Local\Temp\ipykernel_8740\66179513.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data = merged_data.fillna(method='ffill')  # 이전 값으로 채움


In [35]:
import pandas as pd

# 원본 데이터 로드
data = pd.read_csv('../document/corrected_oceandata.csv')

# datetime 형식으로 변환
data['datetime'] = pd.to_datetime(data['datetime'])

# 전체 시간 범위 생성
start_time = data['datetime'].min()
end_time = data['datetime'].max()
full_time_range = pd.date_range(start=start_time, end=end_time, freq='10T')

# 누락된 시간 구간 확인
missing_times = pd.DataFrame({'datetime': full_time_range}).merge(data, on='datetime', how='left')
missing_rows = missing_times[missing_times.isna().any(axis=1)]

print("누락된 시간 구간:")
print(missing_rows['datetime'])



누락된 시간 구간:
Series([], Name: datetime, dtype: datetime64[ns])


C:\Users\user\AppData\Local\Temp\ipykernel_8740\3750417316.py:12: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_time_range = pd.date_range(start=start_time, end=end_time, freq='10T')


In [36]:
import pandas as pd
from datetime import timedelta

# CSV 파일 읽기
ais_data = pd.read_csv('../document/aisdata.csv')
ocean_data = pd.read_csv('../document/corrected_oceandata.csv')

# 시간 데이터 변환 (datetime 형식으로 변환)
ais_data['created_at'] = pd.to_datetime(ais_data['created_at'])
ocean_data['datetime'] = pd.to_datetime(ocean_data['datetime'])

# 병합을 위한 시간 차이 기준 (예: 10분)
time_threshold = timedelta(minutes=10)

# 병합 로직: 가장 가까운 datetime 매칭
def find_closest_with_threshold(row, ocean_df):
    time_differences = (ocean_df['datetime'] - row['created_at']).abs()
    closest_idx = time_differences.idxmin()
    # 시간 차이가 허용 범위 내에 있는 경우에만 반환
    if time_differences.iloc[closest_idx] <= time_threshold:
        return ocean_df.loc[closest_idx]
    return pd.Series(index=ocean_df.columns)  # 허용 범위 초과 시 빈 데이터 반환

# 병합된 결과 저장
merged_data = ais_data.apply(
    lambda row: pd.concat([row, find_closest_with_threshold(row, ocean_data)]),
    axis=1
)

# 결과 저장
merged_data.to_csv('../document/merged_data.csv', index=False, encoding='utf-8-sig')
print("병합된 데이터가 'merged_data.csv'에 저장되었습니다.")


병합된 데이터가 'merged_data.csv'에 저장되었습니다.


In [37]:
# 병합 전 원본 데이터 시간 범위 확인
print(ocean_data['datetime'].min(), ocean_data['datetime'].max())

# 병합 후 데이터 시간 범위 확인
print(merged_data['datetime'].min(), merged_data['datetime'].max())


2025-01-20 00:00:00 2025-01-21 11:20:00
2025-01-20 10:00:00 2025-01-21 11:20:00
